<a href="https://colab.research.google.com/github/andre-arantes/ia/blob/master/tp1/etapa5/MLP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
with open('/content/sample_data/PreProcessamento.pkl', 'rb') as f:
  X_trainscaled, X_testscaled, y_treino, y_teste = pickle.load(f)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = MLPClassifier(max_iter=100,random_state=0)
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
from sklearn.metrics import make_scorer, recall_score

# Create a custom scoring function for recall of class 1
scoring = {'Recall_Class1': make_scorer(recall_score, pos_label=1)}

# Random search of parameters, using 5 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=model, param_distributions=parameter_space,
                              n_iter = 100, scoring=scoring,refit='Recall_Class1',
                              cv = 5, verbose=2, random_state=0, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_trainscaled, y_treino)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 32 is smaller than n_iter=100. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=100, random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.0001, 0.05],
                                        'hidden_layer_sizes': [(10, 30, 10),
                                                               (20,)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']},
                   random_state=0, refit='Recall_Class1',
                   return_train_score=True,
                   scoring={'Recall_Class1': make_scorer(recall_score, pos_label=1)},
                   verbose=2)

**testando o modelo**

In [ ]:
best_model = rf_random.best_estimator_
clf = best_model.predict(X_testscaled)

In [ ]:
print(classification_report(y_teste, clf))
report_MLP = classification_report(y_teste, clf, output_dict=True)
relatorio_MLP = report_MLP['1']['recall']

              precision    recall  f1-score   support

           0       0.99      0.93      0.96     17177
           1       0.38      0.78      0.51       901

    accuracy                           0.93     18078
   macro avg       0.69      0.86      0.74     18078
weighted avg       0.96      0.93      0.94     18078



**Serializar Classification Report**

In [ ]:
import pickle
with open('/content/sample_data/RelatorioMLP.pkl', mode = 'wb') as f:
  pickle.dump([relatorio_MLP], f)